In [3]:
import torch
import numpy as np
import sys
sys.path.append("..")
from Preprocess.preprocessing import *
import torch.nn.functional as F
import torch.optim as optim
from deeprobust.graph.defense import GCN
from deeprobust.graph.utils import *
from deeprobust.graph.data import Dataset
from tqdm import tqdm
import argparse
import random
import copy
import scipy

dataset = 'pubmed'

config = Config(dataset)
config.device = 0
device = config.device

seed = 19
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

data = Data(dataset, dataset+'_ft_norm')

print(data.adj.shape, data.adj.nonzero()[0].shape[0]/2, data.features_norm.shape, data.labels.max(), data.idx_train.shape[0], data.idx_val.shape[0], data.idx_test.shape[0])

data.idx_unlabeled = np.union1d(data.idx_val, data.idx_test)

# Setup Surrogate model
surrogate = GCN(nfeat=data.features.shape[1], nclass=data.labels.max().item()+1,
                            nhid=config.eval_hidden, dropout=config.eval_dropout, weight_decay=config.eval_weight_decay,
                           device=config.device).to(config.device) 

surrogate = surrogate.to(device)
surrogate.fit(data.features_norm, data.adj, data.labels, data.idx_train, data.idx_val, normalize=True)
surrogate.test(data.idx_test)

(19717, 19717) 44325.5 (19717, 500) tensor(2) 1971 1972 15774
Test set results: loss= 0.4365 accuracy= 0.8577


tensor(0.8577, device='cuda:0', dtype=torch.float64)